In [ ]:
!pip install pyopencl

In [ ]:
import pyopencl as cl
import time as tm
import numpy as np

Veamos las plataformas a usar:

In [ ]:
plataforms = cl.get_platforms()
print("Plataformas: ", plataforms)

gpu_devices = plataforms[0].get_devices(cl.device_type.GPU)
print("Devices: ", gpu_devices)

Plataformas:  [<pyopencl.Platform 'NVIDIA CUDA' at 0x3302760>]
Devices:  [<pyopencl.Device 'Tesla T4' on 'NVIDIA CUDA' at 0x3302800>]


Creamos las partes real e imaginarias de los n numeros a probar, en el intervalo $(-2,2)$ tanto para las partes real e imaginarias. Esto porsue se puede demostrar que un numero complejo de módulo mayor a 2 no puede estar dentro del set:

In [ ]:
np.random.seed(0)
n = 200000
real_h = np.random.rand(n).astype(np.float32)*4 - 2
imaginaria_h = np.random.rand(n).astype(np.float32)*4 - 2

Creamos los buffer:

In [ ]:

context = cl.Context(gpu_devices)
queue = cl.CommandQueue(context)

real_d = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=real_h)
imaginaria_d = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=imaginaria_h)
estan_d = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, real_h.nbytes)

Creamos el programa que comprueba si cada punto esta o no en el set:

In [ ]:
code = """
__kernel void check(
  __global float* real_d,
  __global float* imaginaria_d,
  __global float* estan_d)
  {
    int global_id = get_global_id(0);
    int local_id = get_local_id(0);
    int group_id = get_group_id(0); 
    int local_size = get_local_size(0);

    float a = real_d[global_id];
    float b = imaginaria_d[global_id];
    float a_ = real_d[global_id];
    float b_ = imaginaria_d[global_id];
    float temp;
    int iter = 0;
    // mientras el modulo sea menor a 2 (o su cuadrado menor a 4)
    while (iter<10000000 && (a*a + b*b)<4)
    {
      temp = a;
      a = a*a - b*b;
      b = 2*temp*b;
      a += a_;
      b += b_;
      iter++;
    }
    estan_d[global_id] = iter;
  }
"""

In [ ]:
program = cl.Program(context, code).build()

Ejecutamos el programa en la gpu:

In [ ]:
program.check(queue, real_h.shape, None, real_d, imaginaria_d, estan_d)
estan_h = np.empty_like(real_h)
cl.enqueue_copy(queue, estan_h, estan_d)

Calculemos cuales puntos estan dentro, y dividamos eso por el total.

In [ ]:
puntos_dentro = len(list(filter(lambda x: x == 10000000, estan_h)))

area = (puntos_dentro * 16) / n
print(f"El area calculada es de {area:.10f}.") 

El area calculada es de 1.5081600000.


Veamos cual es el error obtenido:

In [ ]:
real = 1.50659177
error = abs(real - area) / real
print(f"Tenemos un error del {error*100:.10f}%.")
 

Tenemos un error del 0.1040912363%.
